In [2]:
%matplotlib widget
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from util import get_path, get_dates_datetime, get_dirname

from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips
from subprocess import call


In [4]:
plate = 22
from directory import directory, run_parallel, find_state, check_state
# directory = "/scratch/shared/mrozemul/Fiji.app/"
listdir = os.listdir(directory) 
list_dir_interest = [name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
dates_datetime = get_dates_datetime(directory,plate)
len(list_dir_interest)

121

In [5]:
begin = 0
end = 120
print('begin =', dates_datetime[begin], '\n  end =', dates_datetime[end])

begin = 2020-12-07 18:49:00 
  end = 2020-12-27 19:50:00


In [6]:
find_state(plate,begin,end)

([(datetime.datetime(2020, 12, 7, 18, 49), 0),
  (datetime.datetime(2020, 12, 7, 22, 49), 1),
  (datetime.datetime(2020, 12, 8, 2, 49), 2),
  (datetime.datetime(2020, 12, 8, 6, 49), 3),
  (datetime.datetime(2020, 12, 8, 10, 50), 4),
  (datetime.datetime(2020, 12, 8, 14, 50), 5),
  (datetime.datetime(2020, 12, 8, 18, 50), 6),
  (datetime.datetime(2020, 12, 8, 22, 50), 7),
  (datetime.datetime(2020, 12, 9, 2, 50), 8),
  (datetime.datetime(2020, 12, 9, 6, 50), 9),
  (datetime.datetime(2020, 12, 9, 11, 23), 10),
  (datetime.datetime(2020, 12, 9, 14, 59), 11),
  (datetime.datetime(2020, 12, 9, 18, 58), 12),
  (datetime.datetime(2020, 12, 9, 22, 58), 13),
  (datetime.datetime(2020, 12, 10, 2, 58), 14),
  (datetime.datetime(2020, 12, 10, 6, 58), 15),
  (datetime.datetime(2020, 12, 10, 11, 4), 16),
  (datetime.datetime(2020, 12, 10, 15, 4), 17),
  (datetime.datetime(2020, 12, 10, 19, 3), 18),
  (datetime.datetime(2020, 12, 10, 23, 3), 19),
  (datetime.datetime(2020, 12, 11, 3, 3), 20),
  (date

In [ ]:
begin = 0
end = 120
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/dilated.mat')
    skel = skel_info['dilated']
    skel_sparse = sparse.lil_matrix(skel)
    call(f'rm {path_snap}/Analysis/dilated.mat',shell=True)
    sio.savemat(path_snap+'/Analysis/dilated.mat',{'dilated' : skel_sparse})

In [ ]:
dates_datetime_chosen = dates_datetime[begin:end+1]
dates = dates_datetime_chosen
for date in dates:
    directory_name = get_dirname(date, plate)
    path_snap = directory+directory_name
    call(f'rm -r {path_snap}',shell=True)

In [ ]:
num_parallel = 120
time = '1:00'
args = [plate]
run_parallel('clean.py',args, begin, end, num_parallel, time, 'clean')